<a href="https://colab.research.google.com/github/girijeshcse/car_finder/blob/nitya/notebooks/Car_model_training_1207.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [5]:
BASE_PATH = '/content/drive/MyDrive/Capstone/Dataset_1/Dataset/'
TRAIN_IMAGES_PATH = os.path.sep.join([BASE_PATH, "Car Images/Train Images"])
TEST_IMAGES_PATH = os.path.sep.join([BASE_PATH, "Car Images/Test Images"])
TRAIN_ANNOTS_PATH = os.path.sep.join([BASE_PATH, "Annotations/Train Annotations.csv"])
TEST_ANNOTS_PATH = os.path.sep.join([BASE_PATH, "Annotations/Test Annotations.csv"])
CAR_NAMES_MAKE_DICT = os.path.sep.join([BASE_PATH, "Car names and make.csv"])

In [6]:
# load the contents of the CSV annotations file
# initialize the list of data (images), our target output predictions
# (bounding box coordinates), along with the filenames of the
# individual images
data = []
targets = []
trainfilenames = []
testfilenames = []
labels = []
bboxes = []
trainimagePaths = []

In [7]:
import pandas as pd 
car_map = pd.read_csv(CAR_NAMES_MAKE_DICT,names=['Cars'])
car_map = car_map.set_index('Cars')

In [8]:
car_map_dict=car_map.to_dict('index')
list(car_map_dict.items())[:5]

[('AM General Hummer SUV 2000', {}),
 ('Acura RL Sedan 2012', {}),
 ('Acura TL Sedan 2012', {}),
 ('Acura TL Type-S 2008', {}),
 ('Acura TSX Sedan 2012', {})]

In [9]:
#fetch a key based on value passed - we need this to build our image paths
print(list(car_map_dict.keys())[173])

Ram C/V Cargo Van Minivan 2012


In [10]:
import csv
with open(TRAIN_ANNOTS_PATH, newline='') as file:
    
    reader = csv.reader(file, delimiter = ' ')
    
    # store the headers in a separate variable,
    # move the reader object to point on the next row
    headings = next(reader)
      
    # output list to store all rows
    Output = []
    for row in reader:
        Output.append(row[:])
  
for row in Output:
    row = str(row)
    row = row.split(",")
    (filename, startX, startY, endX, endY, label) = row
    filename = filename.split("'",1)

In [ ]:
dfRows=[]
for dirname, _, filenames in os.walk('/content/drive/MyDrive/Capstone/Dataset_1/Dataset/Car Images/Train Images'):
  limit=0
  if len(filenames) < 20 or 20 == 0:
    limit=len(filenames)
  else:
    limit=20
  for i in range(0,limit):
    if not os.path.isfile(os.path.join('/content/drive/MyDrive/Capstone/Dataset_1/Dataset/Car Images/Train Images', filenames[i])):
      processedRows=[]
      filename=filenames[i]
      processedRows.append(filename)
      pathSplits=dirname.split('/')
      #print(os.path.join(dirname, filename))
      imagePathtrain=os.path.join(dirname, filename)
      #print(imagePathtrain)
      image = cv2.imread(imagePathtrain)
      (h, w) = image.shape[:2]
      # scale the bounding box coordinates relative to the spatial dimensions of the input image
      startX = float(startX) / w
      startY = float(startY) / h
      endX = float(endX) / w
      endY = float(endY) / h
      # load the image and preprocess it
      image = load_img(imagePathtrain, target_size=(224, 224))
      image_array = img_to_array(image)
      # update our list of data, targets, and filenames
      data = np.append(data, image_array)
      processedRows.append(data)
      processedRows.append(labels)
      bboxes = np.append(bboxes, (startX, startY, endX, endY))
      processedRows.append(bboxes)
      trainimagePaths = np.append(trainimagePaths,imagePathtrain)
      processedRows.append(trainimagePaths)
      dfRows.append(processedRows)
      df=pd.DataFrame(dfRows, columns=["fileName","image_array","data","bboxes","trainimagePaths"]) 

In [ ]:
df.to_csv('/content/drive/MyDrive/Capstone/Dataset_1/Imgarray.csv')           